In [ ]:
!pip install kafka-python


In [ ]:
from kafka import KafkaConsumer
import json
decode = lambda x: json.loads(x.decode('utf-8'))
consumer = KafkaConsumer(bootstrap_servers="slo.swe.th-luebeck.de:9092",group_id= 'groupe_A', auto_offset_reset = "latest") #value_deserializer=decode)
consumer.subscribe(topics = ["quicktest"])

def read_message(consumer):
    for message in consumer:
        print('hallotesettese', message.value)

read_message(consumer)
for message in consumer.poll(max_records=3):
    print('HI', message.value)


In [22]:
#Zum Testen der Datenbankverbindung
import psycopg2

# Verbindung zu Datenbank herstellen
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="swtp_a",
    user="postgres",
    password="12345",
)

# Cursor erstellen
cur = conn.cursor()

# SQL-Befehl zum Einfügen von Daten vorbereiten
insert_query = 'INSERT INTO "Data" (n, ax, ay, az, gx, gy, gz, timestamp) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'

# Testdaten definieren
test_data = {
    "n": 928560,
    "accelerometer": {"x": -0.015625, "y": -0.169678, "z": 1.044434},
    "gyroscope": {"x": -2.251908, "y": -0.916031, "z": -0.389313},
    "timestamp": 111
}

# Daten in die Datenbank einfügen
cur.execute(insert_query, (
    test_data['n'],
    test_data['accelerometer']['x'], test_data['accelerometer']['y'], test_data['accelerometer']['z'],
    test_data['gyroscope']['x'], test_data['gyroscope']['y'], test_data['gyroscope']['z'],
    test_data['timestamp']
))
conn.commit()

# Verbindung schließen
cur.close()
conn.close()


In [ ]:
# Daten aus Kafka in die Datenbank einfügen
import psycopg2
import json
from kafka import KafkaConsumer

# Verbindung zu Datenbank herstellen
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="swtp_a",
    user="postgres",
    password="12345",
)

# Cursor erstellen
cur = conn.cursor()

# SQL-Befehl zum Einfügen von Daten vorbereiten
insert_query = 'INSERT INTO "Data" (n, ax, ay, az, gx, gy, gz, timestamp) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'

# Kafka Consumer erstellen
consumer = KafkaConsumer(
    bootstrap_servers="slo.swe.th-luebeck.de:9092",
    auto_offset_reset="earliest"
) 
consumer.subscribe(topics=["quicktest"])

for message in consumer:
    data = json.loads(message.value)
    
    print(data)
    # Daten in die Datenbank einfügen
    if 'timestamp' in data:
        print('Daten gefunden, füge sie in die DB ein')
        print(data)
        cur.execute(insert_query, (
            data['n'],
            data['ax'], data['ay'], data['az'],
            data['gx'], data['gy'], data['gz'],
            data['timestamp']
        ))
        conn.commit()

# Verbindung schließen
cur.close()
conn.close()

